In [1]:
!pip install selenium pandas beautifulsoup4



Defaulting to user installation because normal site-packages is not writeable


In [2]:
MATCH_URL = "https://www.cricbuzz.com/live-cricket-scorecard/32781/mi-vs-rcb-1st-match-indian-premier-league-2021"
OUTPUT_PATH = "match_data.csv"

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options



In [14]:
def extract_html(url):
    
    options = Options()
    options.add_argument('--headless')  
    options.add_argument('--disable-gpu')  
    options.add_argument('--window-size=1920,1080')

    
    service = Service(executable_path=r"C:\Users\loges\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")

  
    driver = webdriver.Chrome(service=service, options=options)

    print("[INFO] Accessing Cricbuzz...")
    driver.get(url)

    html = driver.page_source
    driver.quit()
    return html

In [15]:
MATCH_URL = "https://www.cricbuzz.com/live-cricket-scorecard/34438/mi-vs-rcb-1st-match-indian-premier-league-2021"
html = extract_html(MATCH_URL)
print(html[:500])  

[INFO] Accessing Cricbuzz...
<html lang="en" itemscope="" itemtype="http://schema.org/WebPage"><head><style>@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"><script type="text/javascript" async="" src="https://sb.scorecardresearch.com/c2/6036484/cs.js"></script><script type="text/javascript" async="" src="http


In [16]:
from bs4 import BeautifulSoup
import pandas as pd


In [17]:
def transform_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    all_overs = soup.find_all("div", class_="cb-col cb-col-100 cb-col cb-com-ln")

    commentary_data = []
    for over in all_overs:
        try:
            over_number = over.find("div", class_="cb-col cb-col-8 text-bold").text.strip()
            ball_description = over.find("div", class_="cb-col cb-col-90 cb-com-ln").text.strip()
            commentary_data.append({"over": over_number, "description": ball_description})
        except Exception as e:
            continue

 
    df = pd.DataFrame(commentary_data)
    return df

In [19]:
df = transform_html(html)
df.head() 

""


In [20]:


def load_to_csv(df, path):
    df.to_csv(path, index=False)
    print(f"[SUCCESS] Data saved to: {path}")


In [21]:
load_to_csv(df, OUTPUT_PATH)


[SUCCESS] Data saved to: match_data.csv


In [28]:
html = extract_html(MATCH_URL)
print(html[:1000]) 


[INFO] Accessing Cricbuzz...
<html lang="en" itemscope="" itemtype="http://schema.org/WebPage"><head><style>@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"><script type="text/javascript" async="" src="https://sb.scorecardresearch.com/c2/6036484/cs.js"></script><script type="text/javascript" async="" src="https://d2r1yp2w7bby2u.cloudfront.net/js/clevertap.min.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-4H06J8XXQH&amp;l=dataLayer&amp;cx=c&amp;gtm=45He54g3h1v71840711za200&amp;tag_exp=102803279~102813109~102887800~102926062~103027016~103051953~103055465~103077950~103106314~103106316"></script><script async="" type="text/javascript" src="https://www.googletagservices.com/tag/js/gpt.js"></script><script type="text/jav

In [40]:
from bs4 import BeautifulSoup
import pandas as pd

def scrape_commentary(html):
    soup = BeautifulSoup(html, "html.parser")
    
    commentary_blocks = soup.select("div.cb-col.cb-col-100.cb-com-ln")
    
    data_list = []
    for block in commentary_blocks:
        try:
            over = block.select_one("span.cb-col.cb-col-8.text-bold").text.strip()
            desc = block.select_one("div.cb-col.cb-col-92.cb-com-ln").text.strip()
            data_list.append({"over": over, "description": desc})
        except AttributeError:
            continue

    return data_list



In [41]:
MATCH_URL = "https://www.cricbuzz.com/live-cricket-scorecard/34438/mi-vs-rcb-1st-match-indian-premier-league-2021"

html = extract_html(MATCH_URL)
data_list = scrape_commentary(html)


[INFO] Accessing Cricbuzz...


In [42]:
df = pd.DataFrame(data_list)
df.to_csv("match_commentary.csv", index=False)
df.head()




""


In [43]:
load_to_csv(df, OUTPUT_PATH)


[SUCCESS] Data saved to: match_data.csv


In [44]:


def run_pipeline():
    print("[START] Running IPL Match ETL Pipeline...\n")

    
    html = extract_html(MATCH_URL)

   
    df = transform_html(html)

  
    load_to_csv(df, OUTPUT_PATH)

    print("\n[COMPLETE] ETL Pipeline finished!")


run_pipeline()


[START] Running IPL Match ETL Pipeline...

[INFO] Accessing Cricbuzz...
[SUCCESS] Data saved to: match_data.csv

[COMPLETE] ETL Pipeline finished!
